# Deriving and Communicating Insights with Data Visualization
## Part 0: Wrangling of Prosper Loan Data
### Jong Min Lee

In [1]:
# import all packages
import pandas as pd
import numpy as np
import copy

## Data Wrangling
`prosperLoanData.csv` which contains the Prosper loan data was made available by [Udacity](https://www.udacity.com/) for download. The file was manually downloaded to the local `data` directory and imported to this project as shown below.

In [2]:
# load .csv file containing the prosper loan data
df = pd.read_csv('data/prosperLoanData.csv')

### 1. Structure of the Dataset
The loan dataset contains 113,937 rows of data for 81 variables, ranging from the `ListingKey` which uniquely identifies each listing posted in [Prosper](www.prosper.com) for the loans requested by borrowers to the `Investors` which indicates the number of investors who funded the loan associated with the listing in Prosper. Review of the key information on and the first five rows of the dataset revealed several aspects of the dataset such as columns with inappropriate data types and/or missing values which render data cleaning necessary. Systematic approach to cleaning the dataset, which includes defining the necessary cleaning operations, coding and performing these operations, and verifying the results are documented in the next two sections.

In [3]:
# summary of the dataframe object
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113937 entries, 0 to 113936
Data columns (total 81 columns):
ListingKey                             113937 non-null object
ListingNumber                          113937 non-null int64
ListingCreationDate                    113937 non-null object
CreditGrade                            28953 non-null object
Term                                   113937 non-null int64
LoanStatus                             113937 non-null object
ClosedDate                             55089 non-null object
BorrowerAPR                            113912 non-null float64
BorrowerRate                           113937 non-null float64
LenderYield                            113937 non-null float64
EstimatedEffectiveYield                84853 non-null float64
EstimatedLoss                          84853 non-null float64
EstimatedReturn                        84853 non-null float64
ProsperRating (numeric)                84853 non-null float64
ProsperRating (Alpha) 

In [4]:
# first five rows of the data
df.head()

,ListingKey,ListingNumber,ListingCreationDate,CreditGrade,Term,LoanStatus,ClosedDate,BorrowerAPR,BorrowerRate,LenderYield,...,LP_ServiceFees,LP_CollectionFees,LP_GrossPrincipalLoss,LP_NetPrincipalLoss,LP_NonPrincipalRecoverypayments,PercentFunded,Recommendations,InvestmentFromFriendsCount,InvestmentFromFriendsAmount,Investors
0,1021339766868145413AB3B,193129,2007-08-26 19:09:29.263000000,C,36,Completed,2009-08-14 00:00:00,0.16516,0.1580,0.1380,...,-133.18,0.0,0.0,0.0,0.0,1.0,0,0,0.0,258
1,10273602499503308B223C1,1209647,2014-02-27 08:28:07.900000000,NaN,36,Current,NaN,0.12016,0.0920,0.0820,...,0.00,0.0,0.0,0.0,0.0,1.0,0,0,0.0,1
2,0EE9337825851032864889A,81716,2007-01-05 15:00:47.090000000,HR,36,Completed,2009-12-17 00:00:00,0.28269,0.2750,0.2400,...,-24.20,0.0,0.0,0.0,0.0,1.0,0,0,0.0,41
3,0EF5356002482715299901A,658116,2012-10-22 11:02:35.010000000,NaN,36,Current,NaN,0.12528,0.0974,0.0874,...,-108.01,0.0,0.0,0.0,0.0,1.0,0,0,0.0,158
4,0F023589499656230C5E3E2,909464,2013-09-14 18:38:39.097000000,NaN,36,Current,NaN,0.24614,0.2085,0.1985,...,-60.27,0.0,0.0,0.0,0.0,1.0,0,0,0.0,20


### 2. Key Features related to a Borrower's APR
This project explores the loan data to identify the correlation of different variables associated with the listings for each loan with a borrower's APR. Insights from this data analysis would be applicable for not only answering such potential questions as whether a loan with a longer term for paying back is associated with a lower APR or if specific categories for which Prosper provides borrowers with its online platform for requesting a loan are associated with a high or low APR but comparing the correlations with the APR across these variables.

To streamline this data analysis, 16 features listed below were selected from the dataset. While few of these columns such as `ListingKey`, `BorrowerState` or `ListingCategory` serve to facilitate the operations for cleaning the dataset or define groups for which further trends in correlations with the APR can be identified, other columns are expected to directly correlate with the APR in differing degrees. For instance, with other conditions being equal, the loan of a borrower who is employed full-time but showed frequent delinquencies may require a higher APR than that for an unemployed borrower with no past delinquency.
* `ListingKey`
* `Term`
* `BorrowerAPR`
* `ListingCategory (name)`
* `BorrowerState`
* `EmploymentStatus`
* `EmploymentstatusDuration`
* `CreditScoreRangeLower`
* `CreditScoreRangeUpper`
* `TotalCreditLinespast7years`
* `TotalInquiries`
* `DelinquenciesLast7Years`
* `BankcardUtilization`
* `DebtToIncomeRatio`
* `StatedMonthlyIncome`
* `LoanOriginalAmount`

A new dataframe object `df_clean` containing only these _main_ features was created from the original dataframe object `df`.

In [5]:
# create sub-dataset from Prosper loan data which contains only the 16 features
df_clean = df.copy()
df_clean = df_clean.iloc[:, np.r_[0, 4, 7, 16, 17, 19, 20, 25, 26, 30, 34, 37, 41, 46, 49, 63]]

### 3. Data Cleaning
#### 3.1 Assessments
Ten observations listed below are those which render data cleaning necessary.
* Data type of the `Term` column is integer, not ordinal categorical variable. \[see _assessments 1_ and _3_\]
* Name of the column, `ListingCategory (numeric)` includes a space. \[see _assessment 1_\]
* Data type of the `ListingCategory (numeric)` column is integer, not nominal categorical variable. \[see _assessment 1_\]
* The `EmploymentStatus` column includes two categories which may indicate the same status of employment: `Not available` and `NaN`. \[see _assessment 4_\]
* Data types of the six columns, `EmploymentStatusDuration`, `CreditScoreRangeLower`, `CreditScoreRangeUpper`, `TotalCreditLinespast7years`, `TotalInquiries`, and `DelinquenciesLast7Years`, are float, not integer as defined in `data_dictionary.xlsx`. \[see _assessments 1_ and _2_\]
* Values under the `StatedMonthlyIncome` column have more than two digits after the decimal although the column represents a monetary amount. \[see _assessment 2_\]
* Data type of the `LoanOriginalAmount` column, which represents a monetary amount, is an integer, not float. \[see _assessments 1_ and _2_\]
* There are 871 duplicate rows of data in the dataframe object `df_clean`. \[see _assessment 5_\]
* 11 of the 16 features in the dataframe object `df_clean` include multiple `NaN` values. \[see _assessments 1_ and _6_\]
* Range of each borrower's credit score is split into lower and upper boundaries although either boundary alone would not provide any statistical significance especially if the difference between the lower and upper boundaries is 19 for all borrowers' credit scores. \[see _assessments 2_ and _7_\]

In [6]:
# assessment 1: summary of the dataframe object
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113937 entries, 0 to 113936
Data columns (total 16 columns):
ListingKey                    113937 non-null object
Term                          113937 non-null int64
BorrowerAPR                   113912 non-null float64
ListingCategory (numeric)     113937 non-null int64
BorrowerState                 108422 non-null object
EmploymentStatus              111682 non-null object
EmploymentStatusDuration      106312 non-null float64
CreditScoreRangeLower         113346 non-null float64
CreditScoreRangeUpper         113346 non-null float64
TotalCreditLinespast7years    113240 non-null float64
TotalInquiries                112778 non-null float64
DelinquenciesLast7Years       112947 non-null float64
BankcardUtilization           106333 non-null float64
DebtToIncomeRatio             105383 non-null float64
StatedMonthlyIncome           113937 non-null float64
LoanOriginalAmount            113937 non-null int64
dtypes: float64(10), int64(3), obj

In [7]:
# assessment 2: first five rows
df_clean.head()

,ListingKey,Term,BorrowerAPR,ListingCategory (numeric),BorrowerState,EmploymentStatus,EmploymentStatusDuration,CreditScoreRangeLower,CreditScoreRangeUpper,TotalCreditLinespast7years,TotalInquiries,DelinquenciesLast7Years,BankcardUtilization,DebtToIncomeRatio,StatedMonthlyIncome,LoanOriginalAmount
0,1021339766868145413AB3B,36,0.16516,0,CO,Self-employed,2.0,640.0,659.0,12.0,3.0,4.0,0.00,0.17,3083.333333,9425
1,10273602499503308B223C1,36,0.12016,2,CO,Employed,44.0,680.0,699.0,29.0,5.0,0.0,0.21,0.18,6125.000000,10000
2,0EE9337825851032864889A,36,0.28269,0,GA,Not available,NaN,480.0,499.0,3.0,1.0,0.0,NaN,0.06,2083.333333,3001
3,0EF5356002482715299901A,36,0.12528,16,GA,Employed,113.0,800.0,819.0,29.0,1.0,14.0,0.04,0.15,2875.000000,10000
4,0F023589499656230C5E3E2,36,0.24614,2,MN,Employed,44.0,680.0,699.0,49.0,9.0,0.0,0.81,0.26,9583.333333,15000


In [8]:
# assessment 3: unique values for Term
df_clean.Term.unique()

array([36, 60, 12], dtype=int64)

In [9]:
# assessment 4: unique values for EmploymentStatus
df_clean.EmploymentStatus.unique()

array(['Self-employed', 'Employed', 'Not available', 'Full-time', 'Other',
       nan, 'Not employed', 'Part-time', 'Retired'], dtype=object)

In [10]:
# assessment 5: number of duplicate rows
df_clean.duplicated().sum()

871

In [11]:
# assessment 6: number of null values in each column
df_clean.isnull().sum()

ListingKey                       0
Term                             0
BorrowerAPR                     25
ListingCategory (numeric)        0
BorrowerState                 5515
EmploymentStatus              2255
EmploymentStatusDuration      7625
CreditScoreRangeLower          591
CreditScoreRangeUpper          591
TotalCreditLinespast7years     697
TotalInquiries                1159
DelinquenciesLast7Years        990
BankcardUtilization           7604
DebtToIncomeRatio             8554
StatedMonthlyIncome              0
LoanOriginalAmount               0
dtype: int64

In [12]:
# assessment 7: range of credit scores
(df_clean['CreditScoreRangeUpper'] - df_clean['CreditScoreRangeLower']).value_counts()

19.0    113346
dtype: int64

#### 3.2 Define, Clean, and Test
For each of the ten assessments documented above, a cleaning operation was defined, coded, performed, and verified.

__Data type of the `Term` column is integer, not ordinal categorical variable.__

Convert the data type of the `Term` column from integer to (ordinal) category.

In [13]:
# ordered list of unique values in Term
terms = list(df_clean.Term.unique())
terms.sort()

# convert data type of Term to (ordered) category
loan_terms = pd.api.types.CategoricalDtype(ordered = True, categories = terms)
df_clean.Term = df_clean.Term.astype(loan_terms)

# verify that data type of Term is (ordered) category
df_clean.Term.dtype

CategoricalDtype(categories=[12, 36, 60], ordered=True)

__Name of the column, `ListingCategory (numeric)` includes a space.__

Change the name of the column from `ListingCategory (numeric)` to `ListingCategory`.

In [14]:
# change column name
df_clean.rename(columns = {'ListingCategory (numeric)':'ListingCategory'}, inplace = True)

# verify that 'ListingCategory', instead of 'ListingCategory (numeric)', is among the available columns
df_clean.columns

Index(['ListingKey', 'Term', 'BorrowerAPR', 'ListingCategory', 'BorrowerState',
       'EmploymentStatus', 'EmploymentStatusDuration', 'CreditScoreRangeLower',
       'CreditScoreRangeUpper', 'TotalCreditLinespast7years', 'TotalInquiries',
       'DelinquenciesLast7Years', 'BankcardUtilization', 'DebtToIncomeRatio',
       'StatedMonthlyIncome', 'LoanOriginalAmount'],
      dtype='object')

__Data type of the `ListingCategory (numeric)` column is integer, not nominal categorical variable.__

Convert the data type of the `ListingCategory` from integer to (nominal) category.

In [15]:
# ordered list of unique values in ListingCategory
categories = list(df_clean.ListingCategory.unique())
categories.sort()

# convert data type of ListingCategory to (nominal) category
listing_categories = pd.api.types.CategoricalDtype(categories = categories)
df_clean.ListingCategory = df_clean.ListingCategory.astype(listing_categories)

# verify that data type of ListingCategory is (nominal) category
df_clean.ListingCategory.dtype

CategoricalDtype(categories=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
                  18, 19, 20],
                 ordered=False)

__The `EmploymentStatus` column includes two categories which may indicate the same status of employment: `Not available` and `NaN`.__

Change all instances of `Not available` in the `EmploymentStatus` column to `NaN`.

In [16]:
# total number of instances of 'Not available' for EmploymentStatus
count_status_before = df_clean.query('EmploymentStatus == "Not available"').shape[0]
count_nan_before = df_clean.EmploymentStatus.isna().sum()

# replace 'Not available' with NaN
df_clean.EmploymentStatus.replace(to_replace = 'Not available', value = np.nan, inplace = True)

# verify that all instances of 'Not available' were replaced with NaN
count_status_after = df_clean.query('EmploymentStatus == "Not available"').shape[0]
count_nan_replaced = df_clean.EmploymentStatus.isna().sum() - count_nan_before
count_status_before, count_status_after, count_nan_replaced

(5347, 0, 5347)

__There are 871 duplicate rows of data in the dataframe object `df_clean`.__

Remove all duplicate rows from the dataframe object `df_clean`.

In [17]:
# remove all duplicate rows
df_clean.drop_duplicates(inplace = True)

# verify that none of the rows is duplicate
df_clean.duplicated().sum()

0

__11 of the 16 features in the dataframe object `df_clean` include multiple `NaN` values.__

Remove all rows from the the dataframe object `df_clean`, which include at least one instance of `NaN`.

In [18]:
# drop all rows with NaN
df_clean.dropna(inplace = True)

# verify that all rows have non-NaN values across columns
df_clean.isnull().sum()

ListingKey                    0
Term                          0
BorrowerAPR                   0
ListingCategory               0
BorrowerState                 0
EmploymentStatus              0
EmploymentStatusDuration      0
CreditScoreRangeLower         0
CreditScoreRangeUpper         0
TotalCreditLinespast7years    0
TotalInquiries                0
DelinquenciesLast7Years       0
BankcardUtilization           0
DebtToIncomeRatio             0
StatedMonthlyIncome           0
LoanOriginalAmount            0
dtype: int64

As a result of dropping rows which either are duplicates or have missing values, the indices of the dataframe object `df_clean` are no longer evenly spaced by a single unit. Reset the index so that the indices increase by 1 from 0.

In [19]:
# indices of the dataframe object being cleaned
df_clean.index

Int64Index([     0,      1,      3,      4,      5,      6,      7,      8,
                10,     12,
            ...
            113927, 113928, 113929, 113930, 113931, 113932, 113933, 113934,
            113935, 113936],
           dtype='int64', length=95381)

In [20]:
# reset index
df_clean.reset_index(drop = True, inplace = True)
df_clean.head()

,ListingKey,Term,BorrowerAPR,ListingCategory,BorrowerState,EmploymentStatus,EmploymentStatusDuration,CreditScoreRangeLower,CreditScoreRangeUpper,TotalCreditLinespast7years,TotalInquiries,DelinquenciesLast7Years,BankcardUtilization,DebtToIncomeRatio,StatedMonthlyIncome,LoanOriginalAmount
0,1021339766868145413AB3B,36,0.16516,0,CO,Self-employed,2.0,640.0,659.0,12.0,3.0,4.0,0.00,0.17,3083.333333,9425
1,10273602499503308B223C1,36,0.12016,2,CO,Employed,44.0,680.0,699.0,29.0,5.0,0.0,0.21,0.18,6125.000000,10000
2,0EF5356002482715299901A,36,0.12528,16,GA,Employed,113.0,800.0,819.0,29.0,1.0,14.0,0.04,0.15,2875.000000,10000
3,0F023589499656230C5E3E2,36,0.24614,2,MN,Employed,44.0,680.0,699.0,49.0,9.0,0.0,0.81,0.26,9583.333333,15000
4,0F05359734824199381F61D,60,0.15425,1,NM,Employed,82.0,740.0,759.0,49.0,2.0,0.0,0.39,0.36,8333.333333,15000


__Data types of the six columns, `EmploymentStatusDuration`, `CreditScoreRangeLower`, `CreditScoreRangeUpper`, `TotalCreditLinespast7years`, `TotalInquiries`, and `DelinquenciesLast7Years`, are float, not integer as defined in `data_dictionary.xlsx`.__

Convert the data types of the six columns from float to integer.

In [21]:
# list of columns the data types of which need to be converted
column_list = ['EmploymentStatusDuration', 'CreditScoreRangeLower', 'CreditScoreRangeUpper', 'TotalCreditLinespast7years', 'TotalInquiries', 'DelinquenciesLast7Years']

# change the data type of each of the six columns to integer
for column in column_list:
    df_clean[column] = df_clean[column].astype(dtype = 'int')
    
    # verify that the data type of the column is integer
    print(column, df_clean[column].dtype)

EmploymentStatusDuration int32
CreditScoreRangeLower int32
CreditScoreRangeUpper int32
TotalCreditLinespast7years int32
TotalInquiries int32
DelinquenciesLast7Years int32


__Values under the `StatedMonthlyIncome` column have more than two digits after the decimal although the column represents a monetary amount.__

Round the values under the `StatedMonthlyIncome` column to two decimal places.

In [22]:
# round StatedMonthlyIncome to two decimal places
df_clean.StatedMonthlyIncome = df_clean.StatedMonthlyIncome.round(decimals = 2)

# verify that StatedMonthlyIncome shows values for two decimal places
df_clean.StatedMonthlyIncome.sample(10)

52355     6133.50
77028    10833.33
41979     9166.67
1804      6583.33
66852     8333.33
31532     9769.42
19384     4833.33
11079     6083.33
43544    11250.00
90977     4166.67
Name: StatedMonthlyIncome, dtype: float64

__Data type of the `LoanOriginalAmount` column, which represents a monetary amount, is an integer, not float.__

Convert the data type of the `LoanOriginalAmount` column from integer to float.

In [23]:
# convert the data type of LoanOriginalAmount to float
df_clean.LoanOriginalAmount = df_clean.LoanOriginalAmount.astype(dtype = 'float')

# verify that the data type of LoanOriginalAmount is float
df_clean.LoanOriginalAmount.dtype

dtype('float64')

__Range of each borrower's credit score is split into lower and upper boundaries.__

* Create a new column `CreditScoreRange` from the existing columns `CreditScoreRangeLower` and `CreditScoreRangeUpper`.
* Change the data type of the `CreditScoreRange` column to (ordinal) category.
* Drop the two columns `CreditScoreRangeLower` and `CreditScoreRangeUpper`.

In [24]:
columns = ['CreditScoreRangeLower', 'CreditScoreRangeUpper']

# convert the data types of existing columns from integer to string
# this conversion is required for the following join operation
for column in columns:
    df_clean[column] = df_clean[column].astype(dtype = 'str')

# join the two boundaries with -
df_clean['CreditScoreRange'] = df_clean[columns].apply(lambda x: '-'.join(x), axis = 1)

# ordered list of unique values in CreditScoreRange
ranges = list(df_clean.CreditScoreRange.unique())
ranges.sort()

# convert data type of CreditScoreRange to (ordinal) category
creditscore_ranges = pd.api.types.CategoricalDtype(categories = ranges)
df_clean.CreditScoreRange = df_clean.ListingCategory.astype(creditscore_ranges)

# verify that data type of CreditScoreRange is (ordinal) category
df_clean.CreditScoreRange.dtype

CategoricalDtype(categories=['520-539', '540-559', '560-579', '580-599', '600-619',
                  '620-639', '640-659', '660-679', '680-699', '700-719',
                  '720-739', '740-759', '760-779', '780-799', '800-819',
                  '820-839', '840-859', '860-879', '880-899'],
                 ordered=False)

In [25]:
# drop two columns
df_clean.drop(columns = columns, inplace = True)

# verify that the two columns for the lower and upper credit scores are no longer available
for column in columns:
    if column not in df_clean.columns:
        print('Column "{}" cannot be found.'.format(column))

Column "CreditScoreRangeLower" cannot be found.
Column "CreditScoreRangeUpper" cannot be found.


#### 3.3 Store Cleaned Dataset

In [26]:
df_clean.to_csv('data/prosperLoanData_clean.csv', index = False)